In [ ]:
!pip install openai --quiet
!pip install langchain --quiet
!pip install cohere --quiet
!pip install tiktoken --quiet
!pip install langchain_community

In [2]:
import os
#os.environ['OPENAI_API_KEY'] = "Your own OPENAI_API_KEY"
#os.environ['COHERE_API_KEY'] = "Your own COHERE_API_KEY"

#Better way
from google.colab import userdata
#os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ['COHERE_API_KEY'] = userdata.get("COHERE_API_KEY")

In [3]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# Step-1: Document Loading

In [4]:
!pip install pypdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 6.7 MB/s eta 0:00:00


In [5]:
from langchain_community.document_loaders import PyPDFLoader

In [6]:
!wget https://raw.githubusercontent.com/sparktalestrainings-sketch/stjoseph0226/b249516622d880bb5308cede494f41d7b2d12bef/Unit3/EMPLOYEE_AGREEMENT.pdf
loader = PyPDFLoader("EMPLOYEE_AGREEMENT.pdf")
pages = loader.load()
print(len(pages))

--2026-02-11 07:17:11--  https://raw.githubusercontent.com/sparktalestrainings-sketch/stjoseph0226/b249516622d880bb5308cede494f41d7b2d12bef/Unit3/EMPLOYEE_AGREEMENT.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206079 (201K) [application/octet-stream]
Saving to: ‘EMPLOYEE_AGREEMENT.pdf’

EMPLOYEE_AGREEMENT. 100%[===================>] 201.25K  --.-KB/s    in 0.02s   

2026-02-11 07:17:11 (7.98 MB/s) - ‘EMPLOYEE_AGREEMENT.pdf’ saved [206079/206079]

13


In [7]:
full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

Pages 13
Lines 369
Words 4204
Charecters 29540


# Step-2:Split the data into Chunks

In [ ]:
!pip install langchain_community

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

133


In [11]:
print(chunks[0])

page_content='EX-10.1 3 smtp_ex10z1.htm EMPLOYEE AGREEMENT
EXHIBIT 10.1
EMPLOYEE AGREEMENT
THIS EMPLOYEE AGREEMENT made as of September___, 2014, by and between SharpSpring,
Inc., a Delaware corporation (the “Company”), whose principal place of business is at 802 NW 5th Avenue,' metadata={'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'creationdate': '2024-04-03T12:51:00+00:00', 'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}


# Step-3: Creating embeddings

In [ ]:
!pip install -U langchain-cohere

In [13]:
#from langchain_community.embeddings import OpenAIEmbeddings, CohereEmbeddings
from langchain_cohere import CohereEmbeddings

In [14]:
#embeddings = OpenAIEmbeddings()
embeddings = CohereEmbeddings(model='embed-english-v3.0', user_agent='langchain')

In [15]:
# A sample embedding

sample_embedding = embeddings.embed_query("You must follow the rules")
print(sample_embedding)

[0.08325195, -0.008575439, -0.02456665, -0.06915283, 0.0016880035, -0.05230713, 0.012207031, 0.014015198, -0.03439331, 0.011100769, -0.01737976, 0.036987305, -0.02947998, -0.0259552, 0.047607422, -0.053863525, 0.006198883, 0.020690918, -0.005748749, -0.025802612, 0.039031982, -0.009643555, -0.046936035, 0.0949707, -0.007926941, 0.02331543, 0.03756714, 0.0013780594, -0.008796692, -0.010513306, 0.01335907, -0.01713562, 0.019058228, 0.05618286, -0.0670166, 0.040405273, -0.013717651, 0.004436493, -0.008506775, 0.038085938, 0.048797607, -0.0046539307, -0.044403076, 0.009353638, -0.057556152, -0.050354004, 0.042510986, 0.002653122, 0.01146698, 0.036743164, 0.029067993, -0.0003402233, 0.026535034, -0.015007019, -0.006374359, -0.0025577545, 0.0030498505, -0.04626465, 0.04763794, -0.021575928, -0.009963989, -0.035217285, 0.05368042, -0.0055236816, 0.05267334, 0.0020389557, -0.024368286, 0.012039185, 0.0074005127, 0.04156494, 0.018310547, -0.07751465, 0.026168823, -0.0395813, 0.008712769, 0.0065

In [16]:
len(sample_embedding)

1024

In [17]:
sample_docs=["You must follow the rules",
            "You must not disclose the rules"]

embeded_vectors1= embeddings.embed_documents(sample_docs)
print(embeded_vectors1)

[[0.07110596, 0.0059547424, -0.01322937, -0.08050537, 0.011558533, -0.040222168, -0.029327393, -0.0126571655, -0.023422241, 0.022750854, 0.0024700165, 0.012901306, -0.036376953, -0.022109985, 0.038879395, -0.03704834, 0.017929077, 0.009391785, -0.01209259, -0.02619934, 0.00082969666, 0.0006737709, -0.054229736, 0.07165527, 0.0107421875, 0.011260986, 0.032348633, -0.0050964355, 0.0032444, -0.012367249, 0.004421234, -0.0020122528, 0.006465912, 0.031280518, -0.053710938, 0.045654297, -0.022399902, 0.011230469, -0.022003174, 0.051727295, 0.02822876, -0.006500244, -0.059326172, -0.007411957, -0.080444336, -0.04547119, 0.037475586, 0.015388489, 0.004081726, 0.024246216, 0.004825592, -0.020004272, 0.013061523, -0.0011978149, -0.014656067, -0.0048675537, -0.009895325, -0.051483154, 0.027145386, -0.01902771, -0.0028743744, -0.033966064, 0.066345215, 0.022277832, 0.033355713, -0.0044555664, -0.021697998, 0.024108887, 0.003282547, 0.03479004, -0.003900528, -0.06774902, 0.011947632, -0.022491455, 

In [18]:
print(len(embeded_vectors1))
print(len(embeded_vectors1[0]))

2
1024


In [19]:
print(sample_embedding)
print(embeded_vectors1[0])

[0.08325195, -0.008575439, -0.02456665, -0.06915283, 0.0016880035, -0.05230713, 0.012207031, 0.014015198, -0.03439331, 0.011100769, -0.01737976, 0.036987305, -0.02947998, -0.0259552, 0.047607422, -0.053863525, 0.006198883, 0.020690918, -0.005748749, -0.025802612, 0.039031982, -0.009643555, -0.046936035, 0.0949707, -0.007926941, 0.02331543, 0.03756714, 0.0013780594, -0.008796692, -0.010513306, 0.01335907, -0.01713562, 0.019058228, 0.05618286, -0.0670166, 0.040405273, -0.013717651, 0.004436493, -0.008506775, 0.038085938, 0.048797607, -0.0046539307, -0.044403076, 0.009353638, -0.057556152, -0.050354004, 0.042510986, 0.002653122, 0.01146698, 0.036743164, 0.029067993, -0.0003402233, 0.026535034, -0.015007019, -0.006374359, -0.0025577545, 0.0030498505, -0.04626465, 0.04763794, -0.021575928, -0.009963989, -0.035217285, 0.05368042, -0.0055236816, 0.05267334, 0.0020389557, -0.024368286, 0.012039185, 0.0074005127, 0.04156494, 0.018310547, -0.07751465, 0.026168823, -0.0395813, 0.008712769, 0.0065

# Step-4: Storing in Vector Stores

In [ ]:
!pip install chromadb -q

In [21]:
from langchain_community.vectorstores import Chroma

In [22]:
emp_rules_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="emp_rules_db"
                          )
emp_rules_db.persist()

/tmp/ipython-input-1712770035.py:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  emp_rules_db.persist()


# Step-5: Retrieval

In [23]:
retriever = emp_rules_db.as_retriever(search_kwargs={'k': 3})
result = retriever.invoke("What is the policy for sick leaves")
print(result)
#i want to display only retrieved text
for doc in result:
    print(doc.page_content)

[Document(metadata={'title': 'EMPLOYEE AGREEMENT', 'creationdate': '2024-04-03T12:51:00+00:00', 'moddate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'page': 1, 'page_label': '2', 'source': 'EMPLOYEE_AGREEMENT.pdf'}, page_content='sick days and personal absence days as authorized by the Company for its other employees.\nARTICLE 6\nFringe Benefits\nEmployee shall be entitled to participate in and receive benefits from all of the Company’s employee\nbenefit plans that are now, or in the future may be, maintained by the'), Document(metadata={'creationdate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'page': 1, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGR

In [24]:
for i in range(len(result)):
  print(result[i].metadata)

{'title': 'EMPLOYEE AGREEMENT', 'creationdate': '2024-04-03T12:51:00+00:00', 'moddate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'page': 1, 'page_label': '2', 'source': 'EMPLOYEE_AGREEMENT.pdf'}
{'creationdate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'page': 1, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '2', 'producer': 'Skia/PDF m123'}
{'creationdate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT', 'producer': 'Skia/PDF m123', 'total_pages': 13, 'source': 'EMPLOYEE_AGREEMENT.pdf',

You can also set a retrieval method that sets a similarity score threshold and only returns documents with a score above that threshold.


### pretty_print_docs

In [25]:
retriever = emp_rules_db.as_retriever(search_kwargs={'k': 3})
result = retriever.invoke("What is the policy for insurance?")
pretty_print_docs(result)

Document 1:

Company for its employees, including, without limitation, the Company’s health insurance plan. No amounts
paid to Employee from an employee benefit plan shall count as compensation due Employee as base salary or
----------------------------------------------------------------------------------------------------
Document 2:

remedies available to the Company with respect to Employee’s breach of his obligations hereunder,
Employee shall defend, indemnify and hold the Company, the Affiliates, and each of their respective
----------------------------------------------------------------------------------------------------
Document 3:

are the Company’s customers, and that all fees and revenues produced from such relationships or contacts are
the exclusive property of the Company. Employee hereby waives and releases all claims and rights of


In [26]:
for i in range(len(result)):
  print(result[i].metadata)

{'total_pages': 13, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'title': 'EMPLOYEE AGREEMENT', 'creationdate': '2024-04-03T12:51:00+00:00', 'moddate': '2024-04-03T12:51:00+00:00', 'page_label': '3', 'producer': 'Skia/PDF m123', 'page': 2}
{'page': 6, 'page_label': '7', 'title': 'EMPLOYEE AGREEMENT', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'creationdate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf'}
{'creationdate': '2024-04-03T12:51:00+00:00', 'producer': 'Skia/PDF m123', 'page': 5, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT', 'total_pages': 13, 'moddate': '2024-04-03T1

In [27]:
retriever = emp_rules_db.as_retriever(search_kwargs={'k': 3})
result=retriever.invoke("Employee Monthly Salary ")
print(result)

[Document(metadata={'total_pages': 13, 'creationdate': '2024-04-03T12:51:00+00:00', 'page_label': '2', 'page': 1, 'title': 'EMPLOYEE AGREEMENT', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}, page_content='Compensation of Employee\n4.1. Base Compensation.  For all services rendered by Employee under this Employee Agreement, the\nCompany agrees to pay Employee the rate of $14,583 per month (the “base salary”), which shall be payable'), Document(metadata={'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'page': 1, 'creationdate': '2024-04-03T12:51:00+00:00', 'page_label': '2', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT', 'producer': 'Skia/PDF m123', 'total_pages': 13}, 

In [28]:
for i in range(len(result)):
  print(result[i].metadata)

{'total_pages': 13, 'creationdate': '2024-04-03T12:51:00+00:00', 'page_label': '2', 'page': 1, 'title': 'EMPLOYEE AGREEMENT', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}
{'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'page': 1, 'creationdate': '2024-04-03T12:51:00+00:00', 'page_label': '2', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT', 'producer': 'Skia/PDF m123', 'total_pages': 13}
{'producer': 'Skia/PDF m123', 'creationdate': '2024-04-03T12:51:00+00:00', 'page_label': '3', 'total_pages': 13, 'page': 2, 'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko